In [ ]:
pip install librosa

     ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
     - -------------------------------------- 0.0/1.0 MB 145.2 kB/s eta 0:00:07
     - -------------------------------------- 0.0/1.0 MB 145.2 kB/s eta 0:00:07
     - -------------------------------------- 0.0/1.0 MB 130.7 kB/s eta 0:00:08
     - -------------------------------------- 0.0/1.0 MB 130.7 kB/s eta 0:00:08
     -- ------------------------------------- 0.1/1.0 MB 163.6 kB/s eta 0:00:06
     --- ------------------------------------ 0.1/1.0 MB 199.1 kB/s eta 0:00:05
     --- ------------------------------------ 0.1/1.0 MB 209.5 kB/s eta 0:00:05
     ---- ----------------------------------- 0.1/1.0 MB 233.8 kB/s eta 0:00:04
     

In [ ]:
pip install resampy

     ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.1 MB 330.3 kB/s eta 0:00:10
     ---------------------------------------- 0.0/3.1 MB 330.3 kB/s eta 0:00:10
      --------------------------------------- 0.0/3.1 MB 196.9 kB/s eta 0:00:16
     - -------------------------------------- 0.1/3.1 MB 353.1 kB/s eta 0:00:09
     - -------------------------------------- 0.1/3.1 MB 409.6 kB/s eta 0:00:08
     - -------------------------------------- 0.1/3.1 MB 426.7 kB/s eta 0:00:07
     -- ------------------------------------- 0.2/3.1 MB 456.4 kB/s eta 0:00:07
     -- ------------------------------------- 0.2/3.1 MB 479.2 kB/s eta 0:00:06
     -- ------------------------------------- 0.2/3.1 MB 492.1 kB/s eta 0:00:06
     --- ------------------------------------ 0.3/3.1 MB 491.5 kB/s eta 0:00:06
     --- ------------------------------------ 0.3/3.1 MB 5

In [ ]:
import os
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# Set the path to the Torgo dataset
dataset_path = r'G:\ML Project\Dataset'

# Function to extract features from audio files
def extract_features(file_path, max_length=200):
    try:
        audio, _ = librosa.load(file_path, res_type='kaiser_fast')
        if len(audio) == 0:
            return None  # Skip empty audio files
        mfccs = librosa.feature.mfcc(y=audio, sr=16000, n_mfcc=40)
        # Pad or truncate to max_length
        if mfccs.shape[1] < max_length:
            pad_width = max_length - mfccs.shape[1]
            mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        else:
            mfccs = mfccs[:, :max_length]
        return np.mean(mfccs, axis=1)  # Use mean values as features
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None


# Load data and labels
data = []
labels = []

for folder in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder)
    if os.path.isdir(folder_path):
        label = folder.split('_')[1]  # Extract gender and condition from folder name
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            if filename.endswith('.wav'):
                features = extract_features(file_path, max_length=200)  # Adjust max_length as needed
                if features is not None:
                    data.append(features)
                    labels.append(label)


Error processing G:\ML Project\Dataset\F_Con\wav_arrayMic_FC01S01_0256.wav: Input signal length=0 is too small to resample from 16000->22050


In [ ]:
#Find the maximum length of the feature vectors
max_length = max(len(features) for features in data if features is not None)

# Pad or truncate all feature vectors to the maximum length
data = [np.pad(features, (0, max_length - len(features)), mode='constant') for features in data if features is not None]

In [ ]:
# Convert data and labels to numpy arrays
data = np.array(data)
labels = np.array(labels)

In [ ]:
# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, encoded_labels, test_size=0.2, random_state=42)

In [ ]:
# Choose a classifier (SVM or Random Forest)
classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
classifier.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [ ]:
# Make predictions on the test set
predictions = classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, predictions)
print(f'Test Accuracy: {accuracy*100}')

Test Accuracy: 98.96265560165975


In [ ]:
classifier2 = SVC(kernel='linear')
classifier2.fit(X_train, y_train)

SVC(kernel='linear')

In [ ]:
predictions2 = classifier2.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, predictions2)
print(f'Test Accuracy: {accuracy*100}')

Test Accuracy: 98.7551867219917


In [ ]:
# Make predictions on the test set
predictions = classifier.predict(X_test)

# Decode the predicted labels
predicted_labels = label_encoder.inverse_transform(predictions)

# Decode the true labels
true_labels = label_encoder.inverse_transform(y_test)

# Display results for class 0
class_0_indices = np.where(true_labels == label_encoder.classes_[0])[0]
class_0_correct = np.sum(predictions[class_0_indices] == y_test[class_0_indices])
class_0_accuracy = class_0_correct / len(class_0_indices) if len(class_0_indices) > 0 else 0

print(f'Class {label_encoder.classes_[0]} Accuracy: {class_0_accuracy}')

# Display results for class 1
class_1_indices = np.where(true_labels == label_encoder.classes_[1])[0]
class_1_correct = np.sum(predictions[class_1_indices] == y_test[class_1_indices])
class_1_accuracy = class_1_correct / len(class_1_indices) if len(class_1_indices) > 0 else 0

print(f'Class {label_encoder.classes_[1]} Accuracy: {class_1_accuracy}')


Class Con Accuracy: 0.9972144846796658
Class Dys Accuracy: 0.959349593495935


In [ ]:
import pandas as pd
# Decode the predicted labels
predicted_labels = label_encoder.inverse_transform(predictions)

# Decode the true labels
true_labels = label_encoder.inverse_transform(y_test)

# Create a DataFrame to store the results
results_df = pd.DataFrame({'True_Labels': true_labels, 'Predicted_Labels': predicted_labels})


# Export the DataFrame to a CSV file
results_df.to_csv('classification_results.csv', index=False)

# Display the first few rows of the results DataFra
results_df

,True_Labels,Predicted_Labels
0,Dys,Dys
1,Dys,Dys
2,Con,Con
3,Con,Con
4,Dys,Dys
...,...,...
477,Dys,Dys
478,Con,Con
479,Con,Con
480,Con,Con
